In [1]:
# utility
import sys
sys.path.append("..")

import utility

# data processing
import pandas as pd
import numpy as np

# modelling keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.regularizers import L1L2, L1, L2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, BinaryCrossentropy

# visualization
import matplotlib.pyplot as plt

In [2]:
data, labels, _ = utility.load_data()

In [3]:
train_data, train_labels, test_data, test_labels, val_data, val_labels, test_true_labels, anom_data = utility.preprocess_data(data, labels)

In [4]:
def build_autoencoder(optimizer, activation, loss, regularizer, dropout_rate):
    # Input image
    input_img = Input(shape=(784,))

    # Encoder
    encoded = Dense(256, activation='relu', kernel_regularizer=regularizer)(input_img)
    encoded = BatchNormalization()(encoded)
    encoded = Dense(128, activation='relu', kernel_regularizer=regularizer)(encoded)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dropout(rate=dropout_rate)(encoded)
    encoded = Dense(32, activation='relu')(encoded)

    # Decoder
    decoded = Dense(32, activation='relu', kernel_regularizer=regularizer)(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dense(64, activation='relu', kernel_regularizer=regularizer)(decoded)
    decoded = Dropout(rate=dropout_rate)(decoded)
    decoded = Dense(128, activation='relu', kernel_regularizer=regularizer)(decoded)
    decoded = Dense(256, activation='relu', kernel_regularizer=regularizer)(decoded)
    decoded = Dense(784, activation=activation)(decoded)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=optimizer, loss=loss)

    return autoencoder

In [5]:
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'activation': ['relu', 'sigmoid', 'tanh'],
    'loss': [MeanSquaredError(), MeanAbsoluteError(), BinaryCrossentropy()],
    'regularizer': [None, L1(1e-5), L2(1e-5), L1(1e-3), L2(1e-3)],
    'dropout_rate': [0.0, 0.1, 0.2],
    'epochs': [100, 150, 200],
    'batch_size': [64, 128, 256]
}

In [6]:
best_model = None
best_loss = float('inf')  # Initialize with a high value

for optimizer in param_grid['optimizer']:
    for activation in param_grid['activation']:
        for loss in param_grid['loss']:
            for regularizer in param_grid['regularizer']:
                for dropout_rate in param_grid['dropout_rate']:
                    for epochs in param_grid['epochs']:
                        for batch_size in param_grid['batch_size']:
                            print(f'Params: {optimizer}, {activation}, {loss}, {regularizer}, {dropout_rate}, {epochs}, {batch_size}')
                            # Build the autoencoder model with current hyperparameters
                            regularizer_instance = None if regularizer is None else regularizer
                            autoencoder = build_autoencoder(optimizer, activation, loss, regularizer_instance, dropout_rate)
                            
                            # Train the model
                            history = autoencoder.fit(train_data, train_data, epochs=epochs, batch_size=batch_size,
                                                      shuffle=True, validation_data=(val_data, val_data),
                                                      callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)], verbose=0)
                            
                            # Evaluate the model on validation data
                            val_loss = autoencoder.evaluate(val_data, val_data, verbose=0)
                            
                            # Check if the current model is better than the best model
                            if val_loss < best_loss:
                                best_loss = val_loss
                                best_model = autoencoder
                                best_params = {
                                    'optimizer': optimizer,
                                    'activation': activation,
                                    'loss': loss,
                                    'regularizer': regularizer,
                                    'dropout_rate': dropout_rate,
                                    'epochs': epochs,
                                    'batch_size': batch_size
                                }


Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 100, 64



Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 100, 128
Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 100, 256
Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 150, 64
Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 150, 128
Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 150, 256
Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 200, 64
Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 200, 128
Params: adam, relu, <keras.src.losses.MeanSquaredError object at 0x000002A4674E1F10>, None, 0.0, 200, 256
Params: adam, relu, <keras.src.losses.MeanSqua

KeyboardInterrupt: 

In [ ]:
# Print the best hyperparameters and corresponding loss
print("Best Hyperparameters:")
print(best_params)
print("Best Validation Loss:", best_loss)